<a href="https://colab.research.google.com/github/20pt32/KLA_Hackaton/blob/main/Milestone4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import rtree
import re
from shapely.geometry import shape, Polygon


def read_file(file_name):
    polygons = []
    with open(file_name) as f:
        lines = f.readlines()
        for line in lines:
            match = re.search(r'xy\s+(\d+)', line)
            if match:
                count = int(match.group(1))
                coords = [list(map(int, x.split())) for x in lines[lines.index(line)+1:lines.index(line)+1+count]]
                polygons.append(coords)
    print(polygons)
    return polygons
    


def search_polygons(search_area_file, template_file):
    search_area = read_file(search_area_file)
    templates = read_file(template_file)

    matches = []
    idx = rtree.index.Index()
    for i, poly in enumerate(search_area):
        poly_shapely = Polygon(poly)
        idx.insert(i, poly_shapely.bounds)
        
    for template in templates:
        template_shapely = Polygon(template)
        for j in idx.intersection(template_shapely.bounds):
            if search_area[j].contains(template_shapely):
                matches.append(template)
    return matches

def write_output(matches, template_file, output_file):
    with open(template_file) as f:
        template_data = f.readlines()
    with open(output_file, 'w') as output:
        for line in template_data:
            if line.startswith('xy'):
                match = re.search(r'(\d+) (\d+)', line)
                if match:
                    match_count = match.group(1)
                    match_coords = [list(map(int, x.split())) for x in matches.pop(0)]
                    output.write(f"xy {match_count} {' '.join([f'{x[0]} {x[1]}' for x in match_coords])}\n")
                else:
                    output.write(line)
            else:
                output.write(line)

def main():
    search_area_file = "Source.txt"
    template_file = "POI.txt"
    output_file = "output.txt"
    matches = search_polygons(search_area_file,template_file,output_file)
    print(matches)
    write_output(matches,template_file,output_file)
    print(matches)